In [2]:
import requests
import json
import pandas as pd
import datetime
import math
import seaborn as sns
import tweepy
import tkinter
from time import sleep
import matplotlib.pyplot as plt
% matplotlib inline

# Pulling Data from Twitter


In [1]:
consumer_key = open("consumer key.txt","r").read()

consumer_secret = open("consumer secret key.txt","r").read()
access_token = open("access token.txt","r").read()
access_token_secret = open("access token secret.txt","r").read()
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

NameError: name 'tweepy' is not defined

In [3]:
import tweepy
import time
from datetime import datetime
import sqlite3


def searchTerm(query, max_tweets):
    conn = sqlite3.connect('twitterprofiles.db')
    c = conn.cursor()

    consumer_key = open("consumer key.txt","r").read()
    consumer_secret = open("consumer secret key.txt","r").read()
    access_token = open("access token.txt","r").read()
    access_token_secret = open("access token secret.txt","r").read()
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    public_tweets = tweepy.Cursor(api.search, q=query).items(max_tweets)
    
    c.execute('CREATE TABLE IF NOT EXISTS '+query+' (dbtweetid int PRIMARY KEY, dbtweetdate text, dbtweettext text, polarity real)')
    
    for tweet in public_tweets:
        tweetid = tweet.id
        tweetdate = tweet.created_at
        tweettext = tweet.text
        polarity = 1
        x.add_row([n,tweetid, tweetdate, tweettext, polarity])
        c.execute("insert into "+query+" values (?,?,?,?)", (tweetid, tweetdate, tweettext, polarity))
        n+=1
        
    conn.commit()
    c.close()

In [150]:
def clearTable(tableName):
    
    conn = sqlite3.connect('twitterprofiles.db')
    c = conn.cursor()
    c.execute(F"DELETE FROM {tableName}")
    conn.commit()
    c.close()
    
clearTable('Trump')


Brandon


In [216]:
def visualizeLocTrends(location):
    Trends =api.trends_place(location)
    Trendsdf = pd.DataFrame(Trends[0]['trends']).sort_values('tweet_volume', ascending=False).head(10)
    # Incomplete

In [217]:
def topicSentVisualization(topic):
    results =  api.search(topic,rpp=100, lang='en', show_user=True)
    df = pd.DataFrame(results)
    # Incomplete

# Sentiment Analysis

In [133]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()